In [91]:
from scipy.stats import t
import numpy as np
import sklearn
from sklearn import cross_validation, datasets, metrics, tree , model_selection, ensemble
from sklearn.linear_model import LinearRegression
import pandas as pd

In [77]:
t = 5
nu = 9.5
scipy.stats.t.cdf(t, nu)

0.9996862011846814

In [78]:
def z(X1, X2, sigma1, sigma2, n):
    return (X1-X2)/np.sqrt((sigma1**2/n)+(sigma2**2/n))

Уровень кальция в крови здоровых молодых женщин равен в среднем 9.5 милиграммам на децилитр и имеет характерное стандартное отклонение 0.4 мг/дл. В сельской больнице Гватемалы для 160 здоровых беременных женщин при первом обращении для ведения беременности был измерен уровень кальция; среднее значение составило 9.57 мг/дл. Можно ли утверждать, что средний уровень кальция в этой популяции отличается от 9.5?

Посчитайте достигаемый уровень значимости. Поскольку известны только среднее и дисперсия, а не сама выборка, нельзя использовать стандартные функции критериев — нужно реализовать формулу достигаемого уровня значимости самостоятельно.

Округлите ответ до четырёх знаков после десятичной точки.

In [79]:
X1 = 9.5
sigma1 = 0.4
n = 160
X2 = 9.57
sigma2 = 0.4
z = z(X1, X2, sigma1, sigma2, n)
print(z)
#stats.norm.cdf(z)

-1.565247584249859


Коллеги, видимо я не понимаю какой-то маленький факт, связанный с заданием 4 и p-value вообще. Судя по тому, что здесь нет обсуждения, этот факт все понятен, а я где-то редкостно ээээ.. не понимаю, назовем это цензурно :)

Возьмем пример с весом детей из лекции и двустороннюю гипотезу. Там тоже нет выборок, но есть стандартное отклонение.

T=(3075-2800)/(500/sqrt(25))=2.75,

p=2(1-St(24 ст.св,2.75))=2(1-0.994426085089858)=0.01114782982028410

в лекиции p=7.1E-13.

Что я делаю не так?

как пойму, что не так, так пойму 4 задание 

In [80]:
data = pd.read_csv("diamonds.txt",delimiter="\t")
print(data)

       carat  depth  table  price     x     y     z
0       0.23   61.5   55.0    326  3.95  3.98  2.43
1       0.21   59.8   61.0    326  3.89  3.84  2.31
2       0.23   56.9   65.0    327  4.05  4.07  2.31
3       0.29   62.4   58.0    334  4.20  4.23  2.63
4       0.31   63.3   58.0    335  4.34  4.35  2.75
5       0.24   62.8   57.0    336  3.94  3.96  2.48
6       0.24   62.3   57.0    336  3.95  3.98  2.47
7       0.26   61.9   55.0    337  4.07  4.11  2.53
8       0.22   65.1   61.0    337  3.87  3.78  2.49
9       0.23   59.4   61.0    338  4.00  4.05  2.39
10      0.30   64.0   55.0    339  4.25  4.28  2.73
11      0.23   62.8   56.0    340  3.93  3.90  2.46
12      0.22   60.4   61.0    342  3.88  3.84  2.33
13      0.31   62.2   54.0    344  4.35  4.37  2.71
14      0.20   60.2   62.0    345  3.79  3.75  2.27
15      0.32   60.9   58.0    345  4.38  4.42  2.68
16      0.30   62.0   54.0    348  4.31  4.34  2.68
17      0.30   63.4   54.0    351  4.23  4.29  2.70
18      0.30

In [143]:
#5
#y = data['price']
#data.pop('price')
X = data
(X_train, 
 X_test, 
 y_train, y_test) = sklearn.model_selection.train_test_split(X, y, 
                                     test_size=0.25, 
                                     random_state=1)
#print(X)

In [144]:
reg_1 = LinearRegression().fit(X_train, y_train)
reg_2 = ensemble.RandomForestRegressor(random_state=1).fit(X_train, y_train)

In [145]:
predict_1 = reg_1.predict(X_test)
predict_2 = reg_2.predict(X_test)
reg_1_score = reg_1.score(X_test, y_test)
reg_2_score = reg_2.score(X_test, y_test)
print(reg_1_score, reg_2_score)

(0.8615514393114848, 0.8720283788117894)


In [146]:
res_1 = abs(predict_1 - y_test.values)
res_2 = abs(predict_2 - y_test.values)

In [147]:
#T=(3075-2800)/(500/sqrt(25))=2.75,

#p=2(1-St(24 ст.св,2.75))=2(1-0.994426085089858)=0.011147829820284

In [148]:
scipy.stats.ttest_rel(res_1, res_2)

Ttest_relResult(statistic=12.930817614948769, pvalue=5.074175575660666e-38)

In [153]:
print(res_1.mean(), res_2.mean())

(890.3764004285604, 803.6143383081302)


In [150]:
res_3 = abs(res_1 - res_2)

In [155]:
from statsmodels.stats.weightstats import *
cm = CompareMeans(DescrStatsW(res_1), DescrStatsW(res_2))
print "95%% confidence interval: [%f, %f]" % cm.tconfint_diff(usevar='unequal')

95% confidence interval: [59.124395, 114.399729]


In [156]:
CompareMeans(DescrStatsW(res_1), DescrStatsW(res_2)).tconfint_diff()

(59.12439539216802, 114.39972884869212)

In [157]:
print "95%% confidence interval: [%f, %f]" % DescrStatsW(res_1 - res_2).tconfint_mean()

95% confidence interval: [73.610088, 99.914037]
